In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 计算信息熵

In [4]:
from math import log
def calcShannonENT(dataset):
    '''
    计算信息熵
    '''
    numEntries = len(dataset)
    labelCounts = {}
    for featVec in dataset:
        currentLabel = featVec[-1]
        if currentLabel not in labelCounts.keys():
            labelCounts[currentLabel] =0
        labelCounts[currentLabel]+=1
    
    shannonEnt =0.0
    for key in labelCounts:
        prob = labelCounts[key] / numEntries
        shannonEnt -=prob * log(prob,2)
    return shannonEnt

In [7]:
def createDataSet():
    dataSet = [[1,1,'yes'],
              [1,1,'yes'],
              [1,0,'no'],
              [1,0,'no'],
              [0,1,'no']]
    return dataSet

In [9]:
mydata = createDataSet()

In [10]:
mydata

[[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [1, 0, 'no'], [0, 1, 'no']]

In [11]:
calcShannonENT(mydata)

0.9709505944546686

In [13]:
#混合信息越多熵越高
mydata[0][-1] = 'maybe'

In [14]:
calcShannonENT(mydata)

1.3709505944546687

## 按指定特征划分数据集

In [19]:
def splitDataSet(dataset,axis,value):
    '''
    按指定特征划分数据集
    '''
    retDataSet = [] 
    for featVect in dataset:
        if(featVect[axis] == value):
            reducedFeatVect = featVect[:axis]
            reducedFeatVect.extend(featVect[axis+1:])
            retDataSet.append(reducedFeatVect)
    return retDataSet

In [20]:
mydata

[[1, 1, 'maybe'], [1, 1, 'yes'], [1, 0, 'no'], [1, 0, 'no'], [0, 1, 'no']]

In [21]:
splitDataSet(mydata,0,1)

[[1, 'maybe'], [1, 'yes'], [0, 'no'], [0, 'no']]

## 选择最好的数据集划分方式